## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np

# Import API key
from config_Copy1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Make sure to run in terminal to turn on gmaps dependencies so figures load
#jupyter nbextension enable --py --sys-prefix widgetsnbextension
#jupyter nbextension enable --py --sys-prefix gmaps

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database_Copy1.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Cherskiy,68.7500,161.3000,35.04,scattered clouds,76,44,22.86,RU,scattered clouds
1,1,Derzhavinsk,51.1000,66.3167,30.85,clear sky,47,0,4.70,KZ,clear sky
2,2,Lao Cai,22.4833,103.9500,74.44,broken clouds,88,68,1.32,VN,broken clouds
3,3,Avarua,-21.2078,-159.7750,80.60,light rain,74,40,11.50,CK,light rain
4,4,Hasaki,35.7333,140.8333,63.00,few clouds,59,20,13.80,JP,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country,Description
3,3,Avarua,-21.2078,-159.7750,80.60,light rain,74,40,11.50,CK,light rain
6,6,Namibe,-15.1961,12.1522,81.05,scattered clouds,77,50,1.63,AO,scattered clouds
15,15,Omboue,-1.5746,9.2618,78.94,overcast clouds,84,96,3.67,GA,overcast clouds
23,23,Atuona,-9.8000,-139.0333,80.49,clear sky,74,2,17.31,PF,clear sky
24,24,Isangel,-19.5500,169.2667,86.00,broken clouds,74,75,5.75,VU,broken clouds
30,30,Harper,4.3750,-7.7169,81.37,broken clouds,80,79,7.63,LR,broken clouds
32,32,Puerto Ayora,-0.7393,-90.3518,82.40,few clouds,74,20,14.97,EC,few clouds
36,36,Malangwa,26.8568,85.5581,78.04,scattered clouds,36,47,9.37,NP,scattered clouds
37,37,Hambantota,6.1241,81.1185,77.79,broken clouds,86,56,8.72,LK,broken clouds
38,38,Rikitea,-23.1203,-134.9692,77.76,scattered clouds,72,33,4.74,PF,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                208
City                   208
Lat                    208
Lng                    208
Max Temp               208
Current Description    208
Humidity               208
Cloudiness             208
Wind Speed             208
Country                208
Description            208
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows present

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,80.60,light rain,-21.2078,-159.7750,
6,Namibe,AO,81.05,scattered clouds,-15.1961,12.1522,
15,Omboue,GA,78.94,overcast clouds,-1.5746,9.2618,
23,Atuona,PF,80.49,clear sky,-9.8000,-139.0333,
24,Isangel,VU,86.00,broken clouds,-19.5500,169.2667,
30,Harper,LR,81.37,broken clouds,4.3750,-7.7169,
32,Puerto Ayora,EC,82.40,few clouds,-0.7393,-90.3518,
36,Malangwa,NP,78.04,scattered clouds,26.8568,85.5581,
37,Hambantota,LK,77.79,broken clouds,6.1241,81.1185,
38,Rikitea,PF,77.76,scattered clouds,-23.1203,-134.9692,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping")

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,80.60,light rain,-21.2078,-159.7750,Paradise Inn
6,Namibe,AO,81.05,scattered clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
15,Omboue,GA,78.94,overcast clouds,-1.5746,9.2618,Hotel Olako
23,Atuona,PF,80.49,clear sky,-9.8000,-139.0333,Villa Enata
24,Isangel,VU,86.00,broken clouds,-19.5500,169.2667,Tanna Lodge
30,Harper,LR,81.37,broken clouds,4.3750,-7.7169,Screensaver
32,Puerto Ayora,EC,82.40,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
36,Malangwa,NP,78.04,scattered clouds,26.8568,85.5581,Prem Meat House
37,Hambantota,LK,77.79,broken clouds,6.1241,81.1185,Bungalow 63
38,Rikitea,PF,77.76,scattered clouds,-23.1203,-134.9692,Pension Maro'i


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,80.60,light rain,-21.2078,-159.7750,Paradise Inn
6,Namibe,AO,81.05,scattered clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
15,Omboue,GA,78.94,overcast clouds,-1.5746,9.2618,Hotel Olako
23,Atuona,PF,80.49,clear sky,-9.8000,-139.0333,Villa Enata
24,Isangel,VU,86.00,broken clouds,-19.5500,169.2667,Tanna Lodge
...,...,...,...,...,...,...,...
711,Porto Velho,BR,77.00,light rain,-8.7619,-63.9039,Hotel Central
712,Yenagoa,NG,79.05,overcast clouds,4.9247,6.2642,Aridolf Resort Wellness and Spa
714,Camacari,BR,75.20,heavy intensity rain,-12.6975,-38.3242,Hotel Cambuci
715,Caborca,MX,90.00,broken clouds,30.5838,-111.1010,Hotel Los Lagos


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# Get the data types
hotel_df.dtypes

City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

hotel_info

['\n<dl>\n<dt>City</dt><dd>Avarua</dd>\n<dt>Country</dt><dd>CK</dd>\n<dt>Current Description</dt><dd>light rain</dd>\n<dt>Max Temp</dt><dd>80.6 °F</dd>\n<dt>Hotel Name</dt><dd>Paradise Inn</dd>\n</dl>\n',
 '\n<dl>\n<dt>City</dt><dd>Namibe</dd>\n<dt>Country</dt><dd>AO</dd>\n<dt>Current Description</dt><dd>scattered clouds</dd>\n<dt>Max Temp</dt><dd>81.05 °F</dd>\n<dt>Hotel Name</dt><dd>Hotel Chik Chik Namibe</dd>\n</dl>\n',
 '\n<dl>\n<dt>City</dt><dd>Omboue</dd>\n<dt>Country</dt><dd>GA</dd>\n<dt>Current Description</dt><dd>overcast clouds</dd>\n<dt>Max Temp</dt><dd>78.94 °F</dd>\n<dt>Hotel Name</dt><dd>Hotel Olako</dd>\n</dl>\n',
 '\n<dl>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Description</dt><dd>clear sky</dd>\n<dt>Max Temp</dt><dd>80.49 °F</dd>\n<dt>Hotel Name</dt><dd>Villa Enata</dd>\n</dl>\n',
 '\n<dl>\n<dt>City</dt><dd>Isangel</dd>\n<dt>Country</dt><dd>VU</dd>\n<dt>Current Description</dt><dd>broken clouds</dd>\n<dt>Max Temp</dt><dd>86.0 °F</dd>\n<d

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
plt.savefig("WeatherPy_vacation_map.png")

<Figure size 432x288 with 0 Axes>